In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect, func ,distinct,inspect
import numpy as np
import pandas as pd
import datetime as dt
import datetime

# Reflect Tables into SQLAlchemy ORM

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime

In [3]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [4]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)
# reflect the tables

In [5]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [6]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [7]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [8]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

### # Get a list of column names and types

In [9]:
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Design a query to retrieve the last 12 months of precipitation data and plot the results

In [10]:
from datetime import datetime
# Maximum date data available in database
max_date_in_db = session.query(func.max(measurement.date)).one()
max_date_in_db = datetime.strptime(max_date_in_db[0], '%Y-%m-%d')
one_year_before_maxdate = datetime(max_date_in_db.year -1, max_date_in_db.month, max_date_in_db.day)
query_result = session.query(measurement.id,measurement.station,measurement.date,
                        measurement.prcp,measurement.tobs).filter(measurement.date 
                                                                  >one_year_before_maxdate).filter(measurement.date 
                                                                  <= max_date_in_db).all()
#for row in query_result:
#    print(row)
df_one_year_data = pd.DataFrame(query_result)
df_one_year_data.head(3)

,id,station,date,prcp,tobs
0,2365,USC00519397,2016-08-24,0.08,79.0
1,2366,USC00519397,2016-08-25,0.08,80.0
2,2367,USC00519397,2016-08-26,0.00,79.0


In [21]:
yy = session.query(func.max(measurement.date)).one()
xx = datetime.strptime(yy[0], '%Y-%m-%d')
type(xx.strftime('%m-%d'))

str

##### Calculate the date 1 year ago from the last data point in the database

In [ ]:
max_date_in_db = session.query(func.max(measurement.date)).one()
max_date_in_db = datetime.strptime(max_date_in_db[0], '%Y-%m-%d')

last_year_date = datetime(max_date_in_db.year -1, max_date_in_db.month, max_date_in_db.day)
print(f'Max Date in database: {max_date_in_db}, Last Year Date From Max Date in DB:{last_year_date}')

#### Perform a query to retrieve the date and precipitation scores

In [ ]:
prcp_query_result = session.query(measurement.date,
                        measurement.prcp).filter(measurement.date 
                                                >one_year_before_maxdate).filter(measurement.date 
                                                <= max_date_in_db).all()
for row in prcp_query_result:
    print(row)

### Save the query results as a Pandas DataFrame and set the index to the date column

In [ ]:
df_query = pd.DataFrame(prcp_query_result)
df_query_date_idx = df_query.set_index('date')
df_query_date_idx.head(5)

### Sort the dataframe by date

In [ ]:
sorted_df_prcp = df_query_date_idx.sort_index()
sorted_df_prcp.head(3)

### Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
prcp_df = sorted_df_prcp.dropna()  #Removed null values
xaxis = prcp_df.index.tolist()
yaxis = prcp_df["prcp"].tolist()
#plt.figure(figsize)
plt.figure(figsize=(20,9))
plt.tick_params(axis='x', which='both',labelbottom=False)  
major_ticks = np.arange(0,365,20)
plt.xticks(major_ticks)
plt.bar(xaxis,yaxis,width = 4 ,color='b',label='Precipitation', align='center')
plt.xlabel("Date")
plt.ylabel("Precipitation")
plt.title(f'Precipitation Data From {last_year_date.year} until {max_date_in_db.year}')
plt.legend()
plt.grid(which='major', axis='both', linestyle='-')
plt.show()

### Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
prcp_df["prcp"].describe()

### Design a query to show how many stations are available in this dataset?

In [ ]:
distinct_count_station_in_dataset= session.query(func.count(distinct(measurement.station))).all()
print(f'Number of Unique stations available in DataSet 12 months record : {distinct_count_station_in_dataset[0]}')

### What are the most active stations? (i.e. what stations have the most rows)?
##### List the stations and the counts in descending order.


In [ ]:
sel = [measurement.station,func.count(measurement.station)]
active_stations = session.query(*sel).group_by(measurement.station).\
                  order_by(func.count(measurement.station).desc()).all()
active_stations

### Using the station id from the previous query, calculate the lowest temperature recorded, 
### highest temperature recorded, and average temperature of the most active station?


In [ ]:
#inspector.get_columns('measurement')

In [ ]:
most_active_station_id = active_stations[0].station
sel = [measurement.station,func.min(measurement.tobs),func.max(measurement.tobs),func.avg(measurement.tobs)]
most_active_station_temp = session.query(*sel).filter(measurement.station==most_active_station_id)
for row in most_active_station_temp:
    print(f'Active Station: {row[0]} Min Temp: {row[1]} Max Temp: {row[2]} Avg Temp: {row[3]}')


### Choose the station with the highest number of temperature observations.


In [ ]:
sel = [measurement.station,func.count(measurement.tobs)]
most_number_of_tobs_station = session.query(*sel).filter \
                              (measurement.date >one_year_before_maxdate).filter\
                              (measurement.date <= max_date_in_db).group_by(measurement.station).\
                              order_by(func.count(measurement.tobs).desc()).first()
most_number_of_tobs_station_id = most_number_of_tobs_station[0]
print(f'Station with Highest Number of tobs : {most_number_of_tobs_station_id}')

####     Query the last 12 months of temperature observation data for this station and plot the results as a histogram

In [ ]:
most_number_of_tobs_station = session.query(measurement.date,measurement.tobs).filter \
                              (measurement.date >one_year_before_maxdate).filter\
                              (measurement.date <= max_date_in_db).filter(measurement.station==most_number_of_tobs_station_id).all()
hist_plot_df = pd.DataFrame(most_number_of_tobs_station)
hist_plot_df.head(3)

In [ ]:
hist_plot = hist_plot_df['tobs'].hist(bins=12,color='blue')
hist_plot.set_title("Temperature Observation")
hist_plot.set_xlabel("Temprature")
hist_plot.set_ylabel("Frequency")

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
        filter(measurement.date >= start_date).filter(measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
df_one_year_data.head(2)

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
#I picked one year data as mention in starting of this notebook as nothing mention about dates in this question
df_start_date = df_one_year_data["date"].min()
df_end_date = df_one_year_data["date"].max()
temp_result = calc_temps(df_start_date, df_end_date)[0]
print(calc_temps(df_start_date, df_end_date))

#### Plot the results from your previous query as a bar chart. 
##### Use "Trip Avg Temp" as your Title
#### Use the average temperature for the y value
#### Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
y_error = temp_result[1] - temp_result[0]
plt.bar(temp_result[0],temp_result[2],yerr=y_error, color='blue')
plt.title('Trip Avg Temp')
plt.ylabel("Temp(F)")
plt.xlim(57,59)
plt.ylim(0,100)


#### Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
#### Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

In [ ]:
sel = [station.station,station.name,station.latitude,station.longitude,
       station.elevation,func.sum(measurement.prcp)]
rainfall_query_result = session.query(*sel).\
        filter(station.station == measurement.station).\
        group_by(measurement.station).\
        filter(measurement.date >= df_start_date).\
        filter(measurement.date <= df_end_date).\
        order_by(func.sum(measurement.prcp).desc()).\
        all()
df_stations = pd.DataFrame(rainfall_query_result, columns=['Station','Name',
                                                           'Latitude','Longitude','Elevation','PrcpSum'])
df_stations

In [34]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", measurement.date) == date).all()
    
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

##### calculate the daily normals for your trip
##### push each tuple of calculations into a list called `normals`
##### Set the start and end date of the trip
##### Use the start and end date to create a range of dates
#### Stip off the year and save a list of %m-%d strings
##### Loop through the list of %m-%d strings and calculate the normals for each date

In [64]:
all_date_query_result = session.query(measurement.id,measurement.station,measurement.date,
                        measurement.prcp,measurement.tobs).all()
df_all_date_query_result = pd.DataFrame(all_date_query_result)
df_hist_date = df_all_date_query_result["date"]
hist_date = pd.to_datetime(df_hist_date).dt.strftime('%m-%d')                                                              
for row in hist_date:
    date_string = row
    hist_day_average = daily_normals(date_string)
    print(f'{date_string} tmin,tmax,tavg: {hist_day_average}')

01-01 Average: [(62.0, 69.15384615384616, 77.0)]
01-02 Average: [(60.0, 69.39622641509433, 77.0)]
01-03 Average: [(62.0, 68.9090909090909, 77.0)]
01-04 Average: [(58.0, 70.0, 76.0)]
01-06 Average: [(61.0, 68.96491228070175, 76.0)]
01-07 Average: [(57.0, 68.54385964912281, 76.0)]
01-08 Average: [(57.0, 67.16071428571429, 75.0)]
01-09 Average: [(58.0, 67.9298245614035, 78.0)]
01-10 Average: [(62.0, 69.74137931034483, 77.0)]
01-11 Average: [(57.0, 67.3103448275862, 78.0)]
01-12 Average: [(57.0, 67.4074074074074, 81.0)]
01-14 Average: [(58.0, 69.52631578947368, 77.0)]
01-15 Average: [(56.0, 69.31372549019608, 78.0)]
01-16 Average: [(54.0, 68.62962962962963, 80.0)]
01-17 Average: [(61.0, 69.07407407407408, 76.0)]
01-18 Average: [(57.0, 68.63157894736842, 77.0)]
01-19 Average: [(60.0, 68.26315789473684, 78.0)]
01-20 Average: [(61.0, 68.86666666666666, 78.0)]
01-21 Average: [(61.0, 70.14545454545454, 76.0)]
01-22 Average: [(60.0, 69.26415094339623, 76.0)]
01-23 Average: [(57.0, 69.50909090909

08-09 Average: [(69.0, 75.98039215686275, 81.0)]
08-10 Average: [(65.0, 76.42, 83.0)]
08-11 Average: [(67.0, 75.98113207547169, 82.0)]
08-12 Average: [(67.0, 76.52727272727273, 83.0)]
08-13 Average: [(71.0, 76.98245614035088, 84.0)]
08-14 Average: [(71.0, 76.77777777777777, 82.0)]
08-15 Average: [(69.0, 76.47169811320755, 83.0)]
08-16 Average: [(69.0, 76.08, 83.0)]
08-17 Average: [(69.0, 76.125, 83.0)]
08-18 Average: [(70.0, 76.92307692307692, 84.0)]
08-19 Average: [(69.0, 75.94642857142857, 85.0)]
08-20 Average: [(68.0, 76.45283018867924, 82.0)]
08-21 Average: [(67.0, 76.4, 83.0)]
08-22 Average: [(69.0, 76.47169811320755, 84.0)]
08-23 Average: [(67.0, 76.6923076923077, 87.0)]
08-24 Average: [(70.0, 75.96153846153847, 84.0)]
08-25 Average: [(69.0, 76.96078431372548, 82.0)]
08-26 Average: [(71.0, 76.56862745098039, 84.0)]
08-27 Average: [(68.0, 76.28301886792453, 84.0)]
08-28 Average: [(67.0, 76.73469387755102, 86.0)]
08-29 Average: [(70.0, 76.23529411764706, 85.0)]
08-30 Average: [(68.

03-02 Average: [(61.0, 70.36666666666666, 78.0)]
03-03 Average: [(60.0, 70.06896551724138, 78.0)]
03-04 Average: [(57.0, 69.27272727272727, 77.0)]
03-05 Average: [(60.0, 67.89090909090909, 76.0)]
03-06 Average: [(60.0, 69.20338983050847, 76.0)]
03-08 Average: [(59.0, 68.72413793103448, 78.0)]
03-09 Average: [(57.0, 68.56140350877193, 75.0)]
03-10 Average: [(57.0, 69.12280701754386, 75.0)]
03-12 Average: [(62.0, 69.88888888888889, 80.0)]
03-13 Average: [(56.0, 69.29629629629629, 77.0)]
03-14 Average: [(62.0, 70.32758620689656, 77.0)]
03-16 Average: [(60.0, 69.70370370370371, 78.0)]
03-17 Average: [(57.0, 68.54, 75.0)]
03-18 Average: [(57.0, 67.7843137254902, 76.0)]
03-19 Average: [(60.0, 69.1, 75.0)]
03-20 Average: [(62.0, 70.11764705882354, 80.0)]
03-21 Average: [(60.0, 71.07692307692308, 77.0)]
03-22 Average: [(65.0, 71.75471698113208, 77.0)]
03-23 Average: [(63.0, 70.79629629629629, 78.0)]
03-24 Average: [(62.0, 71.5, 77.0)]
03-25 Average: [(63.0, 71.5, 78.0)]
03-26 Average: [(64.0, 

10-01 Average: [(69.0, 75.22916666666667, 80.0)]
10-02 Average: [(70.0, 76.26, 83.0)]
10-03 Average: [(66.0, 76.73076923076923, 84.0)]
10-05 Average: [(67.0, 76.16666666666667, 84.0)]
10-06 Average: [(70.0, 75.42, 81.0)]
10-07 Average: [(68.0, 75.6078431372549, 81.0)]
10-08 Average: [(66.0, 76.3265306122449, 86.0)]
10-09 Average: [(69.0, 76.11363636363636, 84.0)]
10-10 Average: [(69.0, 75.85416666666667, 83.0)]
10-11 Average: [(69.0, 76.57142857142857, 84.0)]
10-12 Average: [(65.0, 75.75510204081633, 82.0)]
10-13 Average: [(65.0, 75.98039215686275, 84.0)]
10-14 Average: [(67.0, 75.1923076923077, 82.0)]
10-15 Average: [(67.0, 75.63461538461539, 82.0)]
10-16 Average: [(67.0, 75.59183673469387, 81.0)]
10-17 Average: [(65.0, 75.07843137254902, 83.0)]
10-18 Average: [(65.0, 75.0, 83.0)]
10-19 Average: [(65.0, 73.24489795918367, 81.0)]
10-20 Average: [(67.0, 74.79629629629629, 82.0)]
10-21 Average: [(66.0, 74.96078431372548, 82.0)]
10-22 Average: [(66.0, 74.84313725490196, 81.0)]
10-23 Avera

04-12 Average: [(65.0, 72.22413793103448, 80.0)]
04-13 Average: [(64.0, 71.52542372881356, 79.0)]
04-14 Average: [(65.0, 72.75, 81.0)]
04-15 Average: [(61.0, 71.79310344827586, 79.0)]
04-16 Average: [(62.0, 71.55357142857143, 81.0)]
04-17 Average: [(65.0, 71.93103448275862, 77.0)]
04-18 Average: [(67.0, 72.49152542372882, 77.0)]
04-19 Average: [(65.0, 72.42105263157895, 83.0)]
04-20 Average: [(64.0, 72.73684210526316, 80.0)]
04-21 Average: [(66.0, 72.32142857142857, 78.0)]
04-22 Average: [(65.0, 72.92727272727272, 84.0)]
04-23 Average: [(65.0, 73.01818181818182, 79.0)]
04-24 Average: [(67.0, 73.3103448275862, 80.0)]
04-25 Average: [(66.0, 73.96551724137932, 83.0)]
04-26 Average: [(66.0, 74.01724137931035, 81.0)]
04-27 Average: [(65.0, 72.87272727272727, 81.0)]
04-28 Average: [(63.0, 73.46428571428571, 79.0)]
04-29 Average: [(66.0, 73.30357142857143, 81.0)]
04-30 Average: [(58.0, 71.5576923076923, 78.0)]
05-01 Average: [(65.0, 72.75925925925925, 79.0)]
05-02 Average: [(65.0, 73.05084745

...

10-19 Average: [(65.0, 73.24489795918367, 81.0)]
10-20 Average: [(67.0, 74.79629629629629, 82.0)]
10-21 Average: [(66.0, 74.96078431372548, 82.0)]
10-22 Average: [(66.0, 74.84313725490196, 81.0)]
10-23 Average: [(66.0, 75.74468085106383, 84.0)]
10-24 Average: [(69.0, 74.98039215686275, 81.0)]
10-25 Average: [(68.0, 75.0, 81.0)]
10-26 Average: [(68.0, 75.17307692307692, 81.0)]
10-27 Average: [(64.0, 75.12, 81.0)]
10-28 Average: [(68.0, 74.75471698113208, 82.0)]
10-29 Average: [(68.0, 74.88679245283019, 81.0)]
10-30 Average: [(64.0, 74.66, 81.0)]
10-31 Average: [(67.0, 74.83673469387755, 80.0)]
11-01 Average: [(65.0, 74.86274509803921, 83.0)]
11-02 Average: [(64.0, 74.0625, 81.0)]
11-03 Average: [(66.0, 74.95918367346938, 82.0)]
11-04 Average: [(65.0, 74.0754716981132, 82.0)]
11-05 Average: [(64.0, 73.5576923076923, 78.0)]
11-06 Average: [(66.0, 74.31111111111112, 81.0)]
11-07 Average: [(63.0, 73.76, 80.0)]
11-08 Average: [(67.0, 73.75, 81.0)]
11-09 Average: [(67.0, 73.59615384615384, 81

05-25 Average: [(68.0, 74.27586206896552, 79.0)]
05-26 Average: [(65.0, 74.03636363636363, 82.0)]
05-27 Average: [(65.0, 74.49090909090908, 80.0)]
05-28 Average: [(67.0, 74.27777777777777, 82.0)]
05-29 Average: [(64.0, 74.19607843137256, 80.0)]
05-30 Average: [(69.0, 74.33333333333333, 81.0)]
05-31 Average: [(68.0, 74.4, 81.0)]
06-01 Average: [(64.0, 74.6140350877193, 81.0)]
06-02 Average: [(65.0, 74.4, 81.0)]
06-03 Average: [(65.0, 74.03448275862068, 80.0)]
06-04 Average: [(65.0, 73.81818181818181, 81.0)]
06-05 Average: [(68.0, 74.85454545454546, 81.0)]
06-06 Average: [(70.0, 75.17543859649123, 80.0)]
06-07 Average: [(68.0, 75.07142857142857, 81.0)]
06-08 Average: [(70.0, 75.58620689655173, 81.0)]
06-09 Average: [(69.0, 75.15254237288136, 81.0)]
06-10 Average: [(68.0, 75.14285714285714, 81.0)]
06-12 Average: [(66.0, 75.66666666666667, 83.0)]
06-13 Average: [(68.0, 75.66666666666667, 81.0)]
06-14 Average: [(69.0, 75.62711864406779, 81.0)]
06-15 Average: [(67.0, 75.40350877192982, 81.0)

12-24 Average: [(62.0, 70.0625, 78.0)]
12-25 Average: [(60.0, 71.75, 79.0)]
12-26 Average: [(63.0, 71.44680851063829, 77.0)]
12-27 Average: [(62.0, 71.91666666666667, 79.0)]
12-29 Average: [(57.0, 70.29166666666667, 78.0)]
12-30 Average: [(63.0, 69.86363636363636, 76.0)]
12-31 Average: [(56.0, 68.27906976744185, 77.0)]
01-01 Average: [(62.0, 69.15384615384616, 77.0)]
01-02 Average: [(60.0, 69.39622641509433, 77.0)]
01-03 Average: [(62.0, 68.9090909090909, 77.0)]
01-04 Average: [(58.0, 70.0, 76.0)]
01-05 Average: [(56.0, 67.96428571428571, 76.0)]
01-06 Average: [(61.0, 68.96491228070175, 76.0)]
01-07 Average: [(57.0, 68.54385964912281, 76.0)]
01-08 Average: [(57.0, 67.16071428571429, 75.0)]
01-09 Average: [(58.0, 67.9298245614035, 78.0)]
01-10 Average: [(62.0, 69.74137931034483, 77.0)]
01-11 Average: [(57.0, 67.3103448275862, 78.0)]
01-12 Average: [(57.0, 67.4074074074074, 81.0)]
01-13 Average: [(57.0, 67.25454545454545, 77.0)]
01-14 Average: [(58.0, 69.52631578947368, 77.0)]
01-15 Aver

08-02 Average: [(68.0, 75.60377358490567, 84.0)]
08-03 Average: [(70.0, 76.61111111111111, 85.0)]
08-04 Average: [(69.0, 76.71153846153847, 84.0)]
08-05 Average: [(69.0, 76.14814814814815, 82.0)]
08-06 Average: [(67.0, 76.25, 83.0)]
08-07 Average: [(71.0, 77.15686274509804, 83.0)]
08-08 Average: [(68.0, 76.5576923076923, 83.0)]
08-09 Average: [(69.0, 75.98039215686275, 81.0)]
08-10 Average: [(65.0, 76.42, 83.0)]
08-11 Average: [(67.0, 75.98113207547169, 82.0)]
08-12 Average: [(67.0, 76.52727272727273, 83.0)]
08-13 Average: [(71.0, 76.98245614035088, 84.0)]
08-14 Average: [(71.0, 76.77777777777777, 82.0)]
08-15 Average: [(69.0, 76.47169811320755, 83.0)]
08-16 Average: [(69.0, 76.08, 83.0)]
08-17 Average: [(69.0, 76.125, 83.0)]
08-18 Average: [(70.0, 76.92307692307692, 84.0)]
08-19 Average: [(69.0, 75.94642857142857, 85.0)]
08-20 Average: [(68.0, 76.45283018867924, 82.0)]
08-21 Average: [(67.0, 76.4, 83.0)]
08-22 Average: [(69.0, 76.47169811320755, 84.0)]
08-23 Average: [(67.0, 76.692307

02-27 Average: [(58.0, 69.98148148148148, 77.0)]
02-28 Average: [(65.0, 70.65517241379311, 80.0)]
03-01 Average: [(64.0, 70.0327868852459, 76.0)]
03-02 Average: [(61.0, 70.36666666666666, 78.0)]
03-03 Average: [(60.0, 70.06896551724138, 78.0)]
03-04 Average: [(57.0, 69.27272727272727, 77.0)]
03-05 Average: [(60.0, 67.89090909090909, 76.0)]
03-06 Average: [(60.0, 69.20338983050847, 76.0)]
03-07 Average: [(60.0, 69.17241379310344, 78.0)]
03-08 Average: [(59.0, 68.72413793103448, 78.0)]
03-09 Average: [(57.0, 68.56140350877193, 75.0)]
03-10 Average: [(57.0, 69.12280701754386, 75.0)]
03-11 Average: [(60.0, 69.89285714285714, 82.0)]
03-13 Average: [(56.0, 69.29629629629629, 77.0)]
03-14 Average: [(62.0, 70.32758620689656, 77.0)]
03-15 Average: [(58.0, 68.54716981132076, 76.0)]
03-16 Average: [(60.0, 69.70370370370371, 78.0)]
03-18 Average: [(57.0, 67.7843137254902, 76.0)]
03-19 Average: [(60.0, 69.1, 75.0)]
03-20 Average: [(62.0, 70.11764705882354, 80.0)]
03-21 Average: [(60.0, 71.076923076

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
